In [1]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Create an URL object
url = 'https://www.cbr.ru/eng/hd_base/overnight/?UniDbQuery.Posted=True&UniDbQuery.From=18.06.1998&UniDbQuery.To=04.05.2022'
# Create object page
page = requests.get(url)

# parser-lxml = Change html to Python friendly format
# Obtain page's information
soup = BeautifulSoup(page.text, 'lxml')
soup

# Obtain information from tag <table>
table1 = soup.find(class_ ="data")
table1

# Obtain every title of columns with tag <th>
headers = []
for i in table1.find_all('th'):
 title = i.text
 headers.append(title)
    
# Create a dataframe
mydata = pd.DataFrame(columns = headers)

# Create a for loop to fill mydata
for j in table1.find_all('tr')[1:]:
 row_data = j.find_all('td')
 row = [i.text for i in row_data]
 length = len(mydata)
 mydata.loc[length] = row

#save into a csv file
mydata.to_csv('Interest_rate.csv')


In [2]:
data = pd.read_csv('Interest_rate.csv',index_col=0)
data

,Starting from,Rate
0,04.05.2022,15.0
1,11.04.2022,18.0
2,28.02.2022,21.0
3,14.02.2022,10.5
4,20.12.2021,9.5
...,...,...
106,07.07.1998,85.0
107,01.07.1998,80.0
108,29.06.1998,85.0
109,19.06.1998,60.0


In [3]:
#data frame should be reversed so that index 0 corresponds to the earliest date
data = data[::-1]

#date format should be changed 
from dateutil.parser import parse
def calend(string):
    return parse(string)
data['Calendar_date'] = data['Starting from'].apply(calend)
data = data.drop(['Starting from'],axis=1)
data=data[['Calendar_date','Rate']]
data

/var/folders/hh/b9jhpyvd09x3thlbfkdlhz1c0000gn/T/ipykernel_4433/1902322152.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Calendar_date'] = data['Starting from'].apply(calend)


,Calendar_date,Rate
110,1998-06-18,78.0
109,1998-06-19,60.0
108,1998-06-29,85.0
107,1998-01-07,80.0
106,1998-07-07,85.0
...,...,...
4,2021-12-20,9.5
3,2022-02-14,10.5
2,2022-02-28,21.0
1,2022-11-04,18.0


In [4]:
#the dataframe is not constructed in a consisted manner
#a data frame with all dates and corresponding rate values should be created
date = pd.DataFrame()
date['Calendar_date'] = pd.date_range(start='6/18/1998', end='5/4/2022',freq='B')
date

,Calendar_date
0,1998-06-18
1,1998-06-19
2,1998-06-22
3,1998-06-23
4,1998-06-24
...,...
6225,2022-04-28
6226,2022-04-29
6227,2022-05-02
6228,2022-05-03


In [5]:
#mergind date and data tables
ratedata =  pd.merge(date,data, on="Calendar_date",how='left')
ratedata

,Calendar_date,Rate
0,1998-06-18,78.0
1,1998-06-19,60.0
2,1998-06-22,NaN
3,1998-06-23,NaN
4,1998-06-24,NaN
...,...,...
6225,2022-04-28,NaN
6226,2022-04-29,NaN
6227,2022-05-02,NaN
6228,2022-05-03,NaN


In [7]:
#fill NaN values with previous values
ratedata= ratedata.fillna(method='ffill')
ratedata

,Calendar_date,Rate
0,1998-06-18,78.0
1,1998-06-19,60.0
2,1998-06-22,60.0
3,1998-06-23,60.0
4,1998-06-24,60.0
...,...,...
6225,2022-04-28,15.0
6226,2022-04-29,15.0
6227,2022-05-02,15.0
6228,2022-05-03,15.0
